In [139]:
import os
import chardet
import re
import ast
import pickle
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from pandarallel import pandarallel
from torch.utils.data import Dataset, DataLoader
import torch

pd.set_option("display.max_colwidth", None)
pandarallel.initialize(progress_bar=True, nb_workers=8)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [156]:
DATASET_FILENAME = "movie_conversations.pkl"
MAX_TOKEN_LENGTH = 1024
dataset_generated = False

if os.path.exists(DATASET_FILENAME):
    dataset_generated = True

In [141]:
if not dataset_generated:
    # Detect encoding of the movie lines file
    cwd = os.getcwd()

    with open(cwd + "/movie_lines/movie_lines.txt", "rb") as f:
        result = chardet.detect(f.read())
        movie_lines_encoding = result["encoding"]

    with open(cwd + "/movie_lines/movie_conversations.txt", "rb") as f:
        result = chardet.detect(f.read())
        movie_conversations_encoding = result["encoding"]

    print(movie_lines_encoding)
    print(movie_conversations_encoding)
    # movie_lines_encoding = "Windows-1252"
    # movie_conversations_encoding = "ascii"

In [142]:
if not dataset_generated:
    # Collect individual movie lines
    with open(cwd + "/movie_lines/movie_lines.txt", "r", encoding=movie_lines_encoding) as f:
        content = f.read()

    lines = content.split("\n")

    # Print first 5 lines and verify length is correct
    print(lines[:5])

    # Remove last element of lines because its an empty string
    lines = lines[:-1]
    print(len(lines))

['L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!', 'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!', 'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.', 'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?', "L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go."]
304713


In [143]:
if not dataset_generated:
    # Initialize containers for values to put in dataframe
    line_numbers_dict = {}

    for line in lines:
        # Split on whitespace
        split = line.split(" ")
        line_number = split[0]
        character_id = split[2]

        # Extract the text after the last "+" character
        l = re.split(r'\+\s+(?=[^+]*$)', line)[-1]
        line_numbers_dict[line_number] = (character_id, l)


    # Create dataframe from extracted values
    print(dict(itertools.islice(line_numbers_dict.items(), 10)))

{'L1045': ('u0', 'They do not!'), 'L1044': ('u2', 'They do to!'), 'L985': ('u0', 'I hope so.'), 'L984': ('u2', 'She okay?'), 'L925': ('u0', "Let's go."), 'L924': ('u2', 'Wow'), 'L872': ('u0', "Okay -- you're gonna need to learn how to lie."), 'L871': ('u2', 'No'), 'L870': ('u0', 'I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?'), 'L869': ('u0', 'Like my fear of wearing pastels?')}


In [144]:
if not dataset_generated:
    # Collect movie conversation lists
    with open(cwd + "/movie_lines/movie_conversations.txt", "r", encoding=movie_conversations_encoding) as f:
        content = f.read()

    lines = content.split("\n")

    # Print first 5 lines and verify length is correct
    print(lines[:5])

    # Remove last element of lines because its an empty string
    lines = lines[:-1]
    print(len(lines))

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']", "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']", "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']", "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']", "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']"]
83097


In [145]:
if not dataset_generated:
    # Initialize containers for values to put in dataframe
    speaker1_ids = []
    speaker2_ids = []
    conversation_lines = []

    for line in lines:
        # Split on whitespace
        split = line.split(" ")
        speaker1_ids.append(split[0])
        speaker2_ids.append(split[2])

        # Extract the text after the last "+" character
        l = re.split(r'\+\s+(?=[^+]*$)', line)[-1]
        l = ast.literal_eval(l)
        conversation_lines.append(l)


    # Create dataframe from extracted values
    movie_conversations = pd.DataFrame(list(zip(speaker1_ids, speaker2_ids, conversation_lines)), columns=["speaker1_id", "speaker2_id", "conversation_lines"])
    display(movie_conversations.head())
    movie_conversations.info()

,speaker1_id,speaker2_id,conversation_lines
0,u0,u2,"[L194, L195, L196, L197]"
1,u0,u2,"[L198, L199]"
2,u0,u2,"[L200, L201, L202, L203]"
3,u0,u2,"[L204, L205, L206]"
4,u0,u2,"[L207, L208]"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83097 entries, 0 to 83096
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   speaker1_id         83097 non-null  object
 1   speaker2_id         83097 non-null  object
 2   conversation_lines  83097 non-null  object
dtypes: object(3)
memory usage: 1.9+ MB


In [146]:
# Function for turning movie lines into multi-turn conversations for training
def create_conversation_turns(row):
    speaker1 = row["speaker1_id"]
    conversation_list = row["conversation_lines"]
    convo = ""
    # For each line, add it to the conversation with a role label
    for line_id in conversation_list:
        movie_line = line_numbers_dict[line_id]
        text = movie_line[1]

        if movie_line[0] == speaker1:
            convo += f"<USER>: {text} \n"
        else:
            convo += f"<AGENT>: {text} \n"
        
    return convo

In [147]:
if not dataset_generated:
    # Try it on a sample for testing
    sample = movie_conversations.copy().iloc[:2]
    sample["conversation"] = sample.apply(create_conversation_turns, axis=1)
    display(sample)

    # Call create_conversation_turns on every row of the dataframe
    movie_conversations["conversation"] = movie_conversations.apply(create_conversation_turns, axis=1)

    # Drop speaker id columns because they aren't needed anymore
    movie_conversations.drop(columns=["speaker1_id", "speaker2_id"], inplace=True)

,speaker1_id,speaker2_id,conversation_lines,conversation
0,u0,u2,"[L194, L195, L196, L197]","<USER>: Can we make this quick? Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad. Again. \n<AGENT>: Well, I thought we'd start with pronunciation, if that's okay with you. \n<USER>: Not the hacking and gagging and spitting part. Please. \n<AGENT>: Okay... then how 'bout we try out some French cuisine. Saturday? Night? \n"
1,u0,u2,"[L198, L199]",<USER>: You're asking me out. That's so cute. What's your name again? \n<AGENT>: Forget it. \n


In [148]:
if not dataset_generated:
    display(movie_conversations.head())
    print(movie_conversations["conversation"].head(1).values[0])

,conversation_lines,conversation
0,"[L194, L195, L196, L197]","<USER>: Can we make this quick? Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad. Again. \n<AGENT>: Well, I thought we'd start with pronunciation, if that's okay with you. \n<USER>: Not the hacking and gagging and spitting part. Please. \n<AGENT>: Okay... then how 'bout we try out some French cuisine. Saturday? Night? \n"
1,"[L198, L199]",<USER>: You're asking me out. That's so cute. What's your name again? \n<AGENT>: Forget it. \n
2,"[L200, L201, L202, L203]","<USER>: No, no, it's my fault -- we didn't have a proper introduction --- \n<AGENT>: Cameron. \n<USER>: The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser. My sister. I can't date until she does. \n<AGENT>: Seems like she could get a date easy enough... \n"
3,"[L204, L205, L206]","<AGENT>: Why? \n<USER>: Unsolved mystery. She used to be really popular when she started high school, then it was just like she got sick of it or something. \n<AGENT>: That's a shame. \n"
4,"[L207, L208]","<USER>: Gosh, if only we could find Kat a boyfriend... \n<AGENT>: Let me see what I can do. \n"


<USER>: Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again. 
<AGENT>: Well, I thought we'd start with pronunciation, if that's okay with you. 
<USER>: Not the hacking and gagging and spitting part.  Please. 
<AGENT>: Okay... then how 'bout we try out some French cuisine.  Saturday?  Night? 



In [149]:
if not dataset_generated:
    # Get the token count for each conversation so we can split the ones that are too long
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
    movie_conversations["token_count"] = movie_conversations["conversation"].apply(lambda x: len(tokenizer.encode(x)))
    movie_conversations.head()

Token indices sequence length is longer than the specified maximum sequence length for this model (1335 > 1024). Running this sequence through the model will result in indexing errors


In [150]:
if not dataset_generated:    
    # Get the conversation entries that are over the token limit
    over_token_limit = movie_conversations[movie_conversations["token_count"] > MAX_TOKEN_LENGTH].index
    print(over_token_limit)

Index([11348, 19670, 24318, 42477, 45571, 51994, 53303, 53320, 70355, 70455,
       70670],
      dtype='int64')


In [151]:
if not dataset_generated:
    # NOTE: Be careful not to run this more than once without reseting the dataframe
    new_entries = []

    # For each conversation that is too long, split it in half
    for idx in over_token_limit:
        lines_to_split = movie_conversations.iloc[idx]["conversation_lines"]
        split_idx = (len(lines_to_split)//2)
        first_half = lines_to_split[:split_idx]
        second_half = lines_to_split[split_idx:]

        first_convo = ""
        second_convo = ""
        # For each line, add it to the conversation
        for i, line_id in enumerate(first_half):
            movie_line = line_numbers_dict[line_id]
            text = movie_line[1]

            if i%2 == 0:
                first_convo += f"<USER>: {text} \n"
            else:
                first_convo += f"<AGENT>: {text} \n"

        new_entries.append({"conversation_lines": first_half, "conversation": first_convo})

        for i, line_id in enumerate(second_half):
            movie_line = line_numbers_dict[line_id]
            text = movie_line[1]

            if i%2 == 0:
                second_convo += f"<USER>: {text} \n"
            else:
                second_convo += f"<AGENT>: {text} \n"

        new_entries.append({"conversation_lines": second_half, "conversation": second_convo})

    # Add the new entries from splitting and drop the originals
    movie_conversations = pd.concat([movie_conversations, pd.DataFrame(new_entries)], axis=0, ignore_index=True)
    movie_conversations.drop(index=over_token_limit, inplace=True)

    movie_conversations.reset_index(inplace=True, drop=True)
    movie_conversations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83108 entries, 0 to 83107
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   conversation_lines  83108 non-null  object 
 1   conversation        83108 non-null  object 
 2   token_count         83086 non-null  float64
dtypes: float64(1), object(2)
memory usage: 1.9+ MB


In [152]:
if not dataset_generated:
    # Check that no conversations are over the token limit
    display(movie_conversations[movie_conversations["token_count"] > MAX_TOKEN_LENGTH])

,conversation_lines,conversation,token_count


In [157]:
if not dataset_generated:
    with open(DATASET_FILENAME, "wb") as f:
        print("Writing dataframe to file")
        pickle.dump(movie_conversations, f)
else:
    with open(DATASET_FILENAME, "rb") as f:
        print("Loading dataframe from file")
        movie_conversations = pickle.load(f)

display(movie_conversations.head())
print(movie_conversations["conversation"].head(1).values[0])

Loading dataframe from file


,conversation_lines,conversation,token_count
0,"[L194, L195, L196, L197]","<USER>: Can we make this quick? Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad. Again. \n<AGENT>: Well, I thought we'd start with pronunciation, if that's okay with you. \n<USER>: Not the hacking and gagging and spitting part. Please. \n<AGENT>: Okay... then how 'bout we try out some French cuisine. Saturday? Night? \n",102.0
1,"[L198, L199]",<USER>: You're asking me out. That's so cute. What's your name again? \n<AGENT>: Forget it. \n,32.0
2,"[L200, L201, L202, L203]","<USER>: No, no, it's my fault -- we didn't have a proper introduction --- \n<AGENT>: Cameron. \n<USER>: The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser. My sister. I can't date until she does. \n<AGENT>: Seems like she could get a date easy enough... \n",83.0
3,"[L204, L205, L206]","<AGENT>: Why? \n<USER>: Unsolved mystery. She used to be really popular when she started high school, then it was just like she got sick of it or something. \n<AGENT>: That's a shame. \n",55.0
4,"[L207, L208]","<USER>: Gosh, if only we could find Kat a boyfriend... \n<AGENT>: Let me see what I can do. \n",31.0


<USER>: Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again. 
<AGENT>: Well, I thought we'd start with pronunciation, if that's okay with you. 
<USER>: Not the hacking and gagging and spitting part.  Please. 
<AGENT>: Okay... then how 'bout we try out some French cuisine.  Saturday?  Night? 



In [ ]:
model_name = "gpt2-medium"  # You can specify the desired model size
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

In [ ]:
tokenized_texts = [tokenizer.encode(conversation, return_tensors="pt") for conversation in movie_conversations["conversation"]]

In [ ]:
batch_size = 4
learning_rate = 5e-5
num_epochs = 1

class ConversationDataset(Dataset):
    def __init__(self, tokenized_texts, tokenizer):
        self.tokenized_texts = tokenized_texts
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.tokenized_texts)

    def __getitem__(self, idx):
        return self.tokenized_texts[idx]

dataset = ConversationDataset(tokenized_texts, tokenizer)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for batch in dataloader:
        inputs = batch.to(device)
        labels = inputs.clone()

        optimizer.zero_grad()

        outputs = model(inputs, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")

In [ ]:
def generate_response(prompt, model, tokenizer, max_length=50):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        output = model.generate(input_ids, max_length=max_length)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

# Example usage
prompt = "User: Hello, how are you?"
response = generate_response(prompt, model, tokenizer)
print(response)